# 정규언어에 대한 열거 알고리듬
정규언어(regular language)에 대한 열거 알고리듬(enumeration algorithm)을 작성하는 방법에 대해 생각해 보고
그것을 하스켈로 작성하는 과제를 제시한다.

사실 이미 우리는 `RegExGen` 노트북에서 열거 알고리듬을 염두에 두고 정규식 의미함수를 하스켈로 작성해 보았다.
바람직한 열거 알고리듬에 가까운 형태의 프로그램에 가깝게 다가가기 위해 점진적으로 세 단계에 걸쳐 정규식 의미함수를 개선해 나갔다.
하지만 마지막 세번째 단계에서도 여전히 Kleene star를 처리하는 부분이 미진했다. Kleene Star를 한 단계만 사용하는 경우는
어느 정도 처리가 되었지만 Kleene star가 두 번 이상 중첩되는 경우 여전히 고르게 나열하지 못하는 결과를 얻었던 것을 기억하라.

여기서는 그 마지막 단계에서 시작해서 의미함수 `genRE`를 열거 알고리듬으로 활용할 수 있는 형태로 변형해 보겠다.

## 정규식 데이타 타입 및 도우미 함수
이전에 살펴본 `RegExGen` 노트북에서와 마찬가지 정규식 데이타 타입과 몇 개의 도우미 함수들을 그대로 가져왔다.

In [1]:
data RE -- 정규식 데이타 타입
  = Empty
  | Epsilon
  | Alphabet Char
  | Concat RE RE
  | Union RE RE
  | Kleene RE
  deriving Show

-- 문자열을 Concat으로 이어진 정규식으로 변환해주는 유틸리티 함수
string2re :: String -> RE
string2re "" = Epsilon
string2re s  = foldr1 Concat (map Alphabet s)

import IHaskell.Display

ppRE r = Display [html(formatRE r)]

formatRE Empty = "∅"
formatRE Epsilon = "ε"
formatRE (Alphabet c) = c:[]
formatRE (Concat r1 r2) = formatRE r1 ++ formatRE r2
formatRE (Union r1 r2) = "(" ++ formatRE r1 ++ "+" ++ formatRE r2 ++ ")"
formatRE (Kleene r) = "(" ++ formatRE r ++ ")*"

## Kleene star 연산자의 최대 반복 회수가 제한된 정규식 의미함수
아래는 `RegExGen` 노트북에서 세번째 단계로 작성했던 의미함수 `genRE''`를 이름만 `genRE`로 바꿔서 그대로 옮겨놓았다.
아래 내용을 수정하여 열거 알고리듬이 되도록 해보자. 기본 아이디어는 무한한 회수로 (0회, 1회, 2회, 3회, ...) 반복되는 문자열들의 집합을 유한한 최대 반복 회수를 지정하여 그 일부를 구하는 함수를 작성하는 것이다. 수식 표기법으로 표기해서 예를 들자면 다음과 같다.

$$
\begin{array}{lcl}
\mathcal{L}^0(a{*}) &=& \{ \varepsilon \} \\
\mathcal{L}^1(a{*}) &=& \{ \varepsilon,a \} \\
\mathcal{L}^2(a{*}) &=& \{ \varepsilon,a,aa \} \\
\mathcal{L}^3(a{*}) &=& \{ \varepsilon,a,aa,aaa \} \\
&\vdots&
\end{array}
$$

이를 일반화해서 정리하면 다음과 같다.

$$
\begin{array}{lcl}
\mathcal{L}^0(R{*}) &=& \mathcal{L}(\varepsilon) = \{ \varepsilon \} \\
\mathcal{L}^n(R{*}) &=& \mathcal{L}^{n-1}(\varepsilon + R\cdot(\varepsilon + {R}*))
\end{array}
$$

즉, $\mathcal{L}^n(R{*}) = \mathcal{L}(\varepsilon + R + RR + \ldots + \overbrace{R\cdots R}^{n개})$


위첨자가 붙어 있지 않았던 `RegExGen` 노트북에서 의미함수 $\mathcal{L}$은
최대 반복 회수를 제한한 위첨자가 붙은 의미함수를 이용한 극한으로 정의된고 해석할 수도 있다.

$$
\mathcal{L}(R) = \lim_{n\to\infty} \mathcal{L}^n(R)
$$


여기서 새로 정의하는 `genRE`는 지난번 노트북에서와 달리 추가로 정수 인자를 하나 더 받는 `Int -> RE -> Bool` 타입의 함수이다.

In [2]:
-- 좀더 개선된 merge 함수
-- 길이순으로 우선 선택하고 길이가 같을 경우는 알파벳 사전순으로
-- 두 리스트가 이미 이러한 순서대로 정렬되어 있을 경우
-- 결과 리스트도 마찬가지 순서로 정렬되어 만들어진다
merge [] ys = ys
merge xs [] = xs
merge (x:xs) (y:ys)
  | xlen < ylen = x : merge xs (y:ys)
  | xlen > ylen = y : merge (x:xs) ys
  | x < y       = x : merge xs (y:ys)
  | x > y       = y : merge (x:xs) ys
  | otherwise   = x : merge xs ys
  where
    xlen = length x
    ylen = length y

diagConcProd (x:xs) (y:ys) = (x++y) : ([x++y | x<-xs] `merge` diagConcProd xs ys `merge` [x++y | y<-ys]) 
diagConcProd _ _ = []

replicateRE r 0 = Epsilon
replicateRE r n = foldr1 Concat (replicate n r)

genRE :: Int -> RE -> [String]
genRE _ Empty          = []
genRE _ Epsilon        = [ "" ]
genRE _ (Alphabet c)   = [ c:"" ]
genRE n (Concat r1 r2) = genRE n r1 `diagConcProd` genRE n r2
genRE n (Union r1 r2)  = genRE n r1 `merge` genRE n r2
genRE 0 (Kleene r)     = undefined
genRE n (Kleene r)     = undefined

In [3]:
merge ["","a","bb","aaa"] ["","ab","bb","aba"]

["","a","ab","bb","aaa","aba"]

In [24]:
diagConcProd [replicate n 'a' | n<-[0..8]] [replicate n 'b' | n<-[0..8]]
length $ diagConcProd [replicate n 'a' | n<-[0..8]] [replicate n 'b' | n<-[0..8]]
take 16 $ diagConcProd [replicate n 'a' | n<-[0..]] [replicate n 'b' | n<-[0..]]

["","a","b","aa","ab","bb","aaa","aab","abb","bbb","aaaa","aaab","aabb","abbb","bbbb","aaaaa","aaaab","aaabb","aabbb","abbbb","bbbbb","aaaaaa","aaaaab","aaaabb","aaabbb","aabbbb","abbbbb","bbbbbb","aaaaaaa","aaaaaab","aaaaabb","aaaabbb","aaabbbb","aabbbbb","abbbbbb","bbbbbbb","aaaaaaaa","aaaaaaab","aaaaaabb","aaaaabbb","aaaabbbb","aaabbbbb","aabbbbbb","abbbbbbb","bbbbbbbb","aaaaaaaab","aaaaaaabb","aaaaaabbb","aaaaabbbb","aaaabbbbb","aaabbbbbb","aabbbbbbb","abbbbbbbb","aaaaaaaabb","aaaaaaabbb","aaaaaabbbb","aaaaabbbbb","aaaabbbbbb","aaabbbbbbb","aabbbbbbbb","aaaaaaaabbb","aaaaaaabbbb","aaaaaabbbbb","aaaaabbbbbb","aaaabbbbbbb","aaabbbbbbbb","aaaaaaaabbbb","aaaaaaabbbbb","aaaaaabbbbbb","aaaaabbbbbbb","aaaabbbbbbbb","aaaaaaaabbbbb","aaaaaaabbbbbb","aaaaaabbbbbbb","aaaaabbbbbbbb","aaaaaaaabbbbbb","aaaaaaabbbbbbb","aaaaaabbbbbbbb","aaaaaaaabbbbbbb","aaaaaaabbbbbbbb","aaaaaaaabbbbbbbb"]

81

["","a","b","aa","ab","bb","aaa","aab","abb","bbb","aaaa","aaab","aabb","abbb","bbbb","aaaaa"]

## 테스트

In [5]:
_1 = Alphabet '1'
_0 = Alphabet '0'
_00 = string2re "00"
_01 = string2re "01"
_10 = string2re "10"
_11 = string2re "11"

ppRE Empty
genRE 0 Empty

ppRE Epsilon
genRE 0 Epsilon

ppRE _0
genRE 0 _0

ppRE _1
genRE 0 _1

ppRE (Concat _0 _1)
genRE 0 (Concat _0 _1)

ppRE (Union _0 _1)
genRE 0 (Union _0 _1)

ppRE (Union _00 _11)
genRE 0 (Union _00 _11)

ppRE (Concat (Union _00 _11) (Union _01 _10))
genRE 0 (Concat (Union _00 _11) (Union _01 _10))

∅

[]

ε

[""]

0

...

["0","1"]

(00+11)

["00","11"]

(00+11)(01+10)

["0001","0010","1101","1110"]

In [6]:
ppRE (Kleene _1)
genRE 2 (Kleene _1)
genRE 3 (Kleene _1)

ppRE (Kleene _01)
genRE 2 (Kleene _01)
genRE 3 (Kleene _01)

(1)*

Prelude.undefined

In [7]:
ppRE (Union (Kleene _0) (Kleene _1))
genRE 2 (Union (Kleene _0) (Kleene _1))
genRE 3 (Union (Kleene _0) (Kleene _1))

ppRE (Concat (Kleene _0) (Kleene _1))
genRE 2 (Concat (Kleene _0) (Kleene _1))
genRE 3 (Concat (Kleene _0) (Kleene _1))

((0)*+(1)*)

Prelude.undefined

In [8]:
ppRE (Kleene (Union (Kleene _00) (Kleene _11)))
genRE 2 (Kleene (Union (Kleene _00) (Kleene _11)))
genRE 3 (Kleene (Union (Kleene _00) (Kleene _11)))
genRE 4 (Kleene (Union (Kleene _00) (Kleene _11)))

ppRE (Kleene (Union _00 _11))
genRE (Kleene (Union _00 _11))
genRE (Kleene (Union _00 _11))
genRE (Kleene (Union _00 _11))
genRE (Kleene (Union _00 _11))
genRE (Kleene (Union _00 _11))

(((00)*+(11)*))*

Prelude.undefined

In [9]:
import Data.List (filter, nub, sortBy)

compareLenThenComp x y = case cmplen of { EQ -> compare x y; z -> z }
  where cmplen = compare (length x) (length y)

set1 = filter ((12 >).length) $ sortBy compareLenThenComp $ nub $ genRE 6 (Kleene (Union (Kleene _00) (Kleene _11)))

set2 = filter ((12 >).length) $ sortBy compareLenThenComp $ nub $ genRE 6 (Kleene (Union _00 _11))

set1
set2

set1 == set2

Prelude.undefined

# 과제: 열거 알고리듬으로 정규식에 해당하는 문자열인지 검사

우선 이 과제를 하기 위한 준비작업으로 앞서 다룬 반복회수 제한이 있는 의미함수 `genRE :: Int -> RE -> String`의 정의를 완성하라.
그리고 나서 주어진 문자열이 정규식을 만족하는지 검사하는 함수 `accepts`를 작성하는 것이 이번 과제이다.
또한 `accepts`의 정의를 완성한 다음에는 정규식 10개에 대해 각각 2개 이상의 문자열로 적어도 하나는 만족하고 적어도 하나는 만족하지 않는 문자열로 구성한 20개 이상의 테스트 케이스를 실행한 것을 포함하여 과제를 제출한다.
정규식 10개에는 종합적으로 모든 종류의 정규식 구성 요소 여섯 가지를 두루두루 활용해야 한다. 예를 들면
`Alphabet 'a'`,
`Alphabet 'b'`,
`Alphabet 'c'`,
`Alphabet 'd'`,
`Alphabet 'e'`,
`Alphabet 'f'`,
`Alphabet 'g'`,
`Alphabet 'h'`,
`Alphabet '0'`,
`Alphabet '1'` 이런 식으로 10개를 거의 똑같은 형태의 정규식으로만 해서는 감점요인이 될 것이다.

`accepts`는 반복회수 제한이 있는 의미함수 `genRE`를 이용하면 아주 간단히 정의할 수 있다.
딱 한줄로 20글자 이내로 아래 `undefined`를 대체해 정의할 수 있다.
이 과제의 목적은 복잡한 프로그램을 작성하라는 것이 아니라 이 하스켈 환경에 익숙해지는 것이 목표이다.

문자열이 조건을 만족하는 경우라면 충분히 많이 나열해서 해당 문자열이 나올때까지 열거해 나가면 된다.
그렇게 해당 문자열이 발견되면 우리가 작성하는 `accepts` 함수가 `True`를 돌려주면 되는 것이다.

하지만 일반적으로 어떤 언어에 대한 열거 알고리듬으로는 언어에 속한다고 확신있게 답변할 수 있다는 보장이 없다.
예를 들어 1000개를 나열했을 때 나오지 않았지만 그 다음 100개를 더 나열해 1100개 안에 원하는 문자열이 나올 수도 있지 않은가?
하지만 다행히도 우리가 다루는 정규식은 반복구조가 단순한 편이라 문자열의 길이가 n이라면 반복회수를 n개로 제한한 의미함수로 충분히 필요한 정보를 다 나열할 수 있으며 만약 거기에 나열되어 나오지 않았다면 앞으로 아무리 더 많이 나열하더라도 나올 가망이 없다고 단정지을 수 있다. 이와 관련된 이론적인 성질을 엄밀하게 증명한 것으로 정규식에 대한 **pumping lemma**라는 것이 있으니 이론적으로 심화학습을 하고 싶은 사람들은 찾아보도록.

In [10]:
accepts :: RE -> String -> Bool
accepts r str = undefined